In [1]:
import warnings
warnings.filterwarnings('ignore')

# CHECK WHICH GEMINI MODELS ARE AVAILABLE

# Install/upgrade packages 
!pip install --upgrade google-generativeai langchain-google-genai

In [2]:
# Check what's available
import google.generativeai as genai
import os
from getpass import getpass

# Get API key
if "GOOGLE_API_KEY" not in os.environ:
    api_key = getpass("Enter your Google API Key: ")
    os.environ["GOOGLE_API_KEY"] = api_key

# Configure and list models
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

print("=" * 70)
print("Available Gemini models:")
print("=" * 70)

available_models = []
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        model_name = model.name.replace('models/', '')
        available_models.append(model_name)
        print(f"✅ {model_name}")

print("\n" + "=" * 70)
print(f"Found {len(available_models)} available models")
print("=" * 70)

Available Gemini models:
✅ gemini-2.5-flash
✅ gemini-2.5-pro
✅ gemini-2.0-flash
✅ gemini-2.0-flash-001
✅ gemini-2.0-flash-exp-image-generation
✅ gemini-2.0-flash-lite-001
✅ gemini-2.0-flash-lite
✅ gemini-exp-1206
✅ gemini-2.5-flash-preview-tts
✅ gemini-2.5-pro-preview-tts
✅ gemma-3-1b-it
✅ gemma-3-4b-it
✅ gemma-3-12b-it
✅ gemma-3-27b-it
✅ gemma-3n-e4b-it
✅ gemma-3n-e2b-it
✅ gemini-flash-latest
✅ gemini-flash-lite-latest
✅ gemini-pro-latest
✅ gemini-2.5-flash-lite
✅ gemini-2.5-flash-image
✅ gemini-2.5-flash-preview-09-2025
✅ gemini-2.5-flash-lite-preview-09-2025
✅ gemini-3-pro-preview
✅ gemini-3-flash-preview
✅ gemini-3-pro-image-preview
✅ nano-banana-pro-preview
✅ gemini-robotics-er-1.5-preview
✅ gemini-2.5-computer-use-preview-10-2025
✅ deep-research-pro-preview-12-2025

Found 30 available models


# GOAL ORIENTED PROMPT APPROACH - SUCCESS AND FAILURE CASES

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from getpass import getpass
from IPython.display import display, Markdown

from medical_appointment_agent import (
    get_rate_limit_safe_llm,
    create_task_oriented_agent,
    create_goal_oriented_agent,
    run_task_oriented_workflow,
    create_llm_with_fallback,
    RateLimitError,
)

import time

In [4]:
# Setup available (rate limit safe) LLM from Gemini
api_key = getpass("Enter your Google API Key: ")
os.environ["GOOGLE_API_KEY"] = api_key

llm = get_rate_limit_safe_llm(verbose=True)

🤖 Initializing LLM with model: gemini-2.5-flash


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 45.026210649s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_d

⚠️  Rate limit hit for gemini-2.5-flash


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 38.960197446s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.google

🔄 Trying fallback model: gemini-2.0-flash


...

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 33.904929094s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_d

⚠️  Rate limit hit for gemini-flash-latest
🔄 Trying fallback model: gemini-2.5-pro


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
Please retry in 27.716953051s. [links {
  description: "Learn more about Ge

⚠️  Rate limit hit for gemini-2.5-pro


RateLimitError: All models are rate limited. Please wait and try again. Tried models: gemini-2.5-flash, gemini-2.0-flash, gemini-flash-latest, gemini-2.5-pro

In [ ]:
goal_agent = create_goal_oriented_agent(llm)
# Single goal request
response = goal_agent.invoke({
    "input": "Book me an appointment with a cardiologist"
})
print("\n📋 FINAL RESULT:")
display(Markdown(response["output"]))

# TASK ORIENTED PROMPT APPROACH - SUCCESS AND FAILURE CASES 

In [ ]:
# Setup
# api_key = getpass("Enter your Google API Key: ")
# os.environ["GOOGLE_API_KEY"] = api_key

In [ ]:
## Success Case # 1
# Try with automatic model fallback
try:
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "cardiologists")
    print(result)
    
except RateLimitError as e:
    print(f"\n⚠️  {e}")
    print("💤 Waiting 60 seconds before retrying...")
    time.sleep(60)
    
    # Retry
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "cardiologists")
    print(result)

In [ ]:
## Success Case # 2
result = run_task_oriented_workflow(agent, "heart specialists")
display(Markdown(result))

In [ ]:
## Failure case # 1
# Try with automatic model fallback
try:
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "dermatologist")  # Not available
    
except RateLimitError as e:
    print(f"\n⚠️  {e}")
    print("💤 Waiting 60 seconds before retrying...")
    time.sleep(60)
    
    # Retry
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "dermatologist")  # Not available

display(Markdown(result['output']))

In [ ]:
## Failure case # 2
result = run_task_oriented_workflow(agent, "urologist")      # Not available
display(Markdown(result))